## Parent retrievers

Al fragmentar documentos para su procesamiento y recuperación, a menudo nos enfrentamos a un dilema: 

Por un lado, se podrían preferir documentos más reducidos, de modo que los `embeddings` puedan reflejar su significado de manera más exacta y específica. Cuando un documento es demasiado extenso, existe el riesgo de que los `embeddings` pierdan su significado y precisión.

Por otro lado, es crucial mantener documentos con una longitud considerable para preservar el contexto de cada fragmento, y así garantizar la coherencia e integridad de la información.

`ParentDocumentRetriever` aborda eficazmente esta contradicción al dividir y almacenar fragmentos de datos concisos. Durante el proceso de recuperación, este sistema primero accede a los fragmentos más pequeños y posteriormente identifica y busca los identificadores principales de dichos fragmentos, retornando finalmente los documentos de mayor tamaño. 

Es crucial aclarar que el término "documento principal" hace referencia al documento fuente del que se extrajo un fragmento pequeño. Esto puede ser el documento íntegro original o un segmento más amplio del mismo.

**Ejemplo:**

Por ejemplo, si se está procesando un libro, podríamos querer fragmentar cada capítulo o sección para obtener `embeddings` más precisos sobre los temas tratados en cada uno. En este caso, un capítulo sería un "documento principal", y cada fragmento o sección del capítulo representaría un fragmento más pequeño.

1. **Proceso de Fragmentación:**
   - El libro se divide en capítulos.
   - Cada capítulo se fragmenta en secciones más pequeñas.

2. **Proceso de Recuperación:**
   - `ParentDocumentRetriever` recupera primero las secciones más pequeñas del capítulo.
   - Luego, identifica y recupera el capítulo completo (documento principal) basándose en los fragmentos pequeños.

Este enfoque permite una búsqueda y recuperación de información más eficiente y precisa, asegurando que cada fragmento recuperado mantenga su contexto original y, al mismo tiempo, brinde un entendimiento profundo y detallado de su contenido.


![Parent Retrievers](../diagrams/slide_diagrama_01.png)

## Librerías

In [1]:
from functools import partial

from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import Language, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from src.langchain_docs_loader import LangchainDocsLoader, num_tokens_from_string

load_dotenv()

True

## Funciones de utilidad

In [2]:
get_vectorstore = partial(
    Chroma,
    embedding_function=OpenAIEmbeddings(),
)

## Carga de datos

In [3]:
loader = LangchainDocsLoader()
docs = loader.load()
len(docs)

962

## Recuperación de los documentos completos

In [4]:
child_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.MARKDOWN,
    chunk_size=100,
    chunk_overlap=10,
    length_function=num_tokens_from_string,
)

vectorstore = get_vectorstore(collection_name="full_documents")

store = InMemoryStore()

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

retriever.add_documents(docs)

La cantidad de documentos en nuestra `Store` es igual a la cantidad de documentos en nuestro dataset.

In [5]:
len(list(store.yield_keys()))

962

Al buscar documentos directamente en la `VectorStore`, obtendrás fragmentos de documentos que fueron procesados por el `TextSplitter`.

In [6]:
full_documents_similarity = vectorstore.similarity_search(
    "Does the MultiQueryRetriever might be able to overcome some of the limitations of...?"
)
full_documents_similarity

[Document(page_content='The `MultiQueryRetriever` automates the process of prompt tuning by using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and takes the unique union across all queries to get a larger set of potentially relevant documents. By generating multiple perspectives on the same question, the `MultiQueryRetriever` might be able to overcome some of the limitations of the distance-based retrieval and get a richer set', metadata={'description': 'Distance-based vector database retrieval embeds (represents) queries in high-dimensional space and finds similar embedded documents based on "distance". But, retrieval may produce different results with subtle changes in query wording or if the embeddings do not capture the semantics of the data well. Prompt engineering / tuning is sometimes done to manually address these problems, but can be tedious.', 'doc_id': '82fe97d1-8cd4-4ce

Si ahora realizas una búsqueda en el `ParentDocumentRetriever`, obtendrás los documentos completos.
Esto se debe a que el `ParentDocumentRetriever` primero busca los fragmentos que hacen `match` con la `query`, después busca los documentos completos sin repeticiones y finalmente devuelve el resultado.

In [7]:
full_documents_retriever = retriever.get_relevant_documents(
    "Does the MultiQueryRetriever might be able to overcome some of the limitations of...?"
)
full_documents_retriever

[Document(page_content='# MultiQueryRetriever\n\nDistance-based vector database retrieval embeds (represents) queries in high-dimensional space and finds similar embedded documents based on "distance". But, retrieval may produce different results with subtle changes in query wording or if the embeddings do not capture the semantics of the data well. Prompt engineering / tuning is sometimes done to manually address these problems, but can be tedious.\n\nThe `MultiQueryRetriever` automates the process of prompt tuning by using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and takes the unique union across all queries to get a larger set of potentially relevant documents. By generating multiple perspectives on the same question, the `MultiQueryRetriever` might be able to overcome some of the limitations of the distance-based retrieval and get a richer set of results.\n\n```python\n# Bu

Puedes corroborar que el `ParentDocumentRetriever` está regresando el subconjunto `único` de documentos completos al comparar el número de documentos recuperados por el `VectorStore` y el `ParentDocumentRetriever`.

In [8]:
[doc.metadata["source"] for doc in full_documents_similarity], [
    doc.metadata["source"] for doc in full_documents_retriever
]

(['https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever',
  'https://python.langchain.com/docs/integrations/retrievers/merger_retriever',
  'https://python.langchain.com/docs/use_cases/question_answering/',
  'https://python.langchain.com/docs/modules/data_connection/'],
 ['https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever',
  'https://python.langchain.com/docs/integrations/retrievers/merger_retriever',
  'https://python.langchain.com/docs/use_cases/question_answering/',
  'https://python.langchain.com/docs/modules/data_connection/'])

## Recuperación de fragmentos largos en lugar de documentos completos

Los documentos pueden ser muy grandes para ser recuperados en su totalidad y ser útiles. 

Por ejemplo, un documento completo podría ser un libro, pero quizá sólo necesito un capítulo para responder a mi pregunta. O quizá sólo necesito un par de párrafos.

Si planeas utilizar los documentos recuperados en un proceso de `Retrival Augmented Generation` (RAG), es posible que los documentos gigantes ni siquiera puedan ser procesados por la ventana de contexto del modelo de lenguaje.

Para este caso, el `ParentDocumentRetriever` puede ser configurado para romper los documentos en fragmentos pequeños, buscar sobre ellos y luego devolver fragmentos más largos (sin ser el documento completo).

In [9]:
parent_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.MARKDOWN,
    chunk_size=400,
    chunk_overlap=40,
    length_function=num_tokens_from_string,
)

child_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.MARKDOWN,
    chunk_size=100,
    chunk_overlap=10,
    length_function=num_tokens_from_string,
)

vectorstore = get_vectorstore(collection_name="big_fragments")

store = InMemoryStore()

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

retriever.add_documents(docs)

Ahora hay más documentos en el `Store` dado que cada documento se ha dividido en fragmentos más pequeños.

In [10]:
len(list(store.yield_keys()))

6473

In [11]:
vectorstore.similarity_search(
    "Does the MultiQueryRetriever might be able to overcome some of the limitations of...?"
)

[Document(page_content='The `MultiQueryRetriever` automates the process of prompt tuning by using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and takes the unique union across all queries to get a larger set of potentially relevant documents. By generating multiple perspectives on the same question, the `MultiQueryRetriever` might be able to overcome some of the limitations of the distance-based retrieval and get a richer set', metadata={'description': 'Distance-based vector database retrieval embeds (represents) queries in high-dimensional space and finds similar embedded documents based on "distance". But, retrieval may produce different results with subtle changes in query wording or if the embeddings do not capture the semantics of the data well. Prompt engineering / tuning is sometimes done to manually address these problems, but can be tedious.', 'doc_id': '44da0302-d174-4fe

In [12]:
retriever.get_relevant_documents(
    "Does the MultiQueryRetriever might be able to overcome some of the limitations of...?"
)

[Document(page_content='# MultiQueryRetriever\n\nDistance-based vector database retrieval embeds (represents) queries in high-dimensional space and finds similar embedded documents based on "distance". But, retrieval may produce different results with subtle changes in query wording or if the embeddings do not capture the semantics of the data well. Prompt engineering / tuning is sometimes done to manually address these problems, but can be tedious.\n\nThe `MultiQueryRetriever` automates the process of prompt tuning by using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and takes the unique union across all queries to get a larger set of potentially relevant documents. By generating multiple perspectives on the same question, the `MultiQueryRetriever` might be able to overcome some of the limitations of the distance-based retrieval and get a richer set of results.\n\n```python\n# Bu